---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    # Return desired percentage:
    return spam_data[spam_data['target']==1].shape[0] / spam_data.shape[0] * 100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    # Fit the CountVectorizer to the training data:
    count_vec = CountVectorizer().fit(X_train)
    # Retrieve list of tokens:
    tokens = count_vec.get_feature_names()
    # Create comparison variable:
    length = 0
    # Store longest word:
    longest = ''
    # Loop over list to get longest word:
    for token in tokens:
        # Update if longer:
        if len(token) > length:
            length = len(token)
            longest = token
    
    return longest

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    ## Pre-processing ##
    
    # Create CountVectorizer object:
    count_vec = CountVectorizer()
    # Fit and transform on training data:
    X_train_transf = count_vec.fit_transform(X_train)
    # Transform test data:
    X_test_transf = count_vec.transform(X_test)
    
    ## ML Model ##
    
    # Create and fit multinomial naive bayes object:
    multi_nb = MultinomialNB(alpha=0.1).fit(X_train_transf, y_train)
    
    ## Model Evaluation ##
    
    # Obtain probability for transformed test data:
    proba_test = multi_nb.predict_proba(X_test_transf)
    # Calculate AUC:
    auc = roc_auc_score(y_test, proba_test[:,1]) #use only probability of being class 1 to match expected value
    # Return model AUC:
    return auc

In [8]:
answer_three()

0.99154542213469599

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    # Create tfidf model:
    tfidf = TfidfVectorizer()
    # Fit and transform training data:
    X_train_vectorized = tfidf.fit_transform(X_train)
    
    # Get feature names:
    feature_names = tfidf.get_feature_names()
    # Get idf values:
    idf_values = tfidf.idf_
    
    # Store values in dataframe:
    df_tfidf = pd.DataFrame({"feature name": feature_names, "tf-idf": idf_values})
    # Sort values and keep top 20:
    smallest = df_tfidf.sort_values(['tf-idf', 'feature name']).head(20)
    largest = df_tfidf.sort_values(['tf-idf', 'feature name'], ascending=[False, True]).head(20)
    # Set feature name as index:
    smallest.set_index('feature name', inplace=True)
    largest.set_index('feature name', inplace=True)
    
    return smallest, largest

In [25]:
answer_four()

(                tf-idf
 feature name          
 to            2.198406
 you           2.265645
 the           2.707383
 in            2.890761
 and           2.976764
 is            3.003012
 me            3.111530
 for           3.206840
 it            3.222174
 my            3.231044
 call          3.297812
 your          3.300196
 of            3.319473
 have          3.354130
 that          3.408477
 on            3.463136
 now           3.465949
 can           3.545053
 are           3.560414
 so            3.566625,                 tf-idf
 feature name          
 000pes        8.644919
 0089          8.644919
 0121          8.644919
 01223585236   8.644919
 0125698789    8.644919
 02072069400   8.644919
 02073162414   8.644919
 02085076972   8.644919
 021           8.644919
 0430          8.644919
 07008009200   8.644919
 07099833605   8.644919
 07123456789   8.644919
 0721072       8.644919
 07753741225   8.644919
 077xxx        8.644919
 078           8.644919
 07808247860   8

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [26]:
def answer_five():
    ## Pre-processing ##
    
    # Create tfidf model:
    tfidf = TfidfVectorizer()
    # Fit and transform data:
    X_train_transf = tfidf.fit_transform(X_train)
    X_test_transf = tfidf.transform(X_test)
    
    ## ML Model ##
    
    # Create and fit multinomial naive bayes object:
    multi_nb = MultinomialNB(alpha=0.1).fit(X_train_transf, y_train)
    
    ## Model Evaluation ##
    
    # Obtain probability for transformed test data:
    proba_test = multi_nb.predict_proba(X_test_transf)
    # Calculate AUC:
    auc = roc_auc_score(y_test, proba_test[:,1]) #use only probability of being class 1 to match expected value
    # Return model AUC:
    return auc

In [27]:
answer_five()

0.99470740030219185

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [ ]:
def answer_six():
    
    
    return #Your answer here

In [ ]:
answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.svm import SVC

def answer_seven():
    
    
    return #Your answer here

In [ ]:
answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [ ]:
def answer_eight():
    
    
    return #Your answer here

In [ ]:
answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    
    return #Your answer here

In [ ]:
answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
def answer_ten():
    
    
    return #Your answer here

In [ ]:
answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
def answer_eleven():
    
    
    return #Your answer here

In [ ]:
answer_eleven()